In [1]:
import pandas as pd
from GPTModel import GPTInference

In [2]:
df = pd.read_csv('/home/sanjana/explainable_factual_evaluation/datasets/fact_annotated/old_model_annotated/dialogue_aggrefact.csv')

df.head()

,Unnamed: 0,DocID,Dialogue,Model,Summary,w/ Error,CorefE,CorefE_text,CircE,CircE_text,...,GramE,GramE_text,PredE,PredE_text,SubjObjE,SubjObjE_text,OtherE,OtherE_text,LinkE,LinkE_text
0,0,13809941,Thelma: i dont have anything to wear\nLouisa: ...,human_ref,Louisa will lend Thelma her red velvet dress.,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1,1,13809941,Thelma: i dont have anything to wear\nLouisa: ...,bart_large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,2,13809941,Thelma: i dont have anything to wear\nLouisa: ...,mv-bart_large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,3,13809941,Thelma: i dont have anything to wear\nLouisa: ...,co-ref bart large,Thelma doesn't have anything to wear. Louisa w...,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
4,4,13809941,Thelma: i dont have anything to wear\nLouisa: ...,condigsum bart large,Louisa will bring Thelma her red velvet dress.,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [3]:
def get_atomic_facts_gpt(gpt_model_atomic, model, text, text_type):
    instr = f'Segment the following {text_type} into important facts'
    prompt = f'{instr}\nDialogue: {text}'
    print(prompt)
    gpt_response = gpt_model_atomic.get_chatgpt_response(prompt, model = model)
    return gpt_response

In [4]:
import spacy
nlp = spacy.load("en_core_web_sm")
def mask_all_errtypes(text):
    doc = nlp(text)
    origin_tokens = [token for token in doc]
    all_masked_results = []
    mask_idx = []
    mask_tokens = []
    for idx, token in enumerate(doc):
        append = False
        if 'subj' in token.dep_ or 'obj' in token.dep_:
            # print(idx, origin_tokens)
            # masked_text = [each.text for each in origin_tokens[:idx]] + ['<BLANK>'] + [each.text for each in origin_tokens[idx+1:]]
            # masked_text = ' '.join(masked_text)
            # print(masked_text)
            append = True
            token_type = 'subjobj'
            
        elif 'VERB' == token.pos_:
            append = True 
            token_type = 'predicate'

        elif 'ADV' == token.pos_:
            append = True
            token_type = 'circumstance'

        elif 'ADP' == token.pos_:
            append = True
            token_type = 'circumstance'

        elif 'PRON' == token.pos_:
            append = True
            token_type = 'coreference'

        

        if append:
            mask_idx.append(idx)
            mask_tokens.append((idx, token.text, token_type))
    # print([])
    print(mask_tokens)  

    error_types_indices_map = {}
    for token_idx, token_text, err_type in mask_tokens:
        if err_type not in error_types_indices_map:
            error_types_indices_map[err_type] = []
        error_types_indices_map[err_type] += [token_idx]
        
    for err_type, err_indices in error_types_indices_map.items():
        for idx in err_indices:
            masked_text = [each.text for each in origin_tokens[:idx]] + ['<BLANK>'] + [each.text for each in origin_tokens[idx+1:]]
            masked_text = ' '.join(masked_text)
            all_masked_results.append((masked_text, origin_tokens[idx].text, err_type))
    return all_masked_results

2023-09-01 06:54:00.902848: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-01 06:54:02.076583: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
# def get_subjobj_answer(gpt_model_corruptor, dlg, text, answer):
#     instruction = "List other subject or object entities from the dialogue that can be used to fill in the <BLANK> in the summary"
#     # instruction = "List subject or object entities that can be used to fill in the <BLANK>"
#     prompt_template = f'Dialogue: {dlg}\nSummary: {text}\n{instruction}\nAnswer: {answer},'
#     # print(prompt_template)
#     # print('--')
#     # prompt_template = f'Summary: {text}\n{instruction}\nAnswer: {answer},'
#     answer = gpt_model_corruptor.get_chatgpt_response(prompt_template)
#     return answer

def get_predicate_answer(gpt_model_corruptor, dlg, atomic_facts, summary, answer):
    instruction_verb = f'What alternative words or phrases to "{answer}" can be used to fill in the <BLANK> in the summary that would make the summary inconsistent or misleading with respect to the dialogue or facts.'
    instruction_tense = f'What different tenses of verbs or phrases of the word {answer} that can be used to fill in the <BLANK> in the summary that would make the summary inconsistent or misleading with respect to the dialogue or facts.'
    instruction_negation = f"What words or phrases that negate {answer} and can be used to fill in the <BLANK> in the summary that would make the summary inconsistent or misleading with respect to the dialogue or facts."
    answer_all = []
    
    prompt_template_verb = f'Dialogue: {dlg}\nFacts: {atomic_facts}\nSummary: {summary}\n{instruction}\nAnswer:'
    answer_verb = gpt_model_corruptor.get_chatgpt_response(prompt_template_verb)
    print('ANSWER VERB', answer_verb)
    if ',' in answer_verb:
        answer_all += answer_verb.split(',')
    
    prompt_template_tense  = f'Dialogue: {dlg}\nFacts: {atomic_facts}\nSummary: {summary}\n{instruction}\nAnswer:'
    answer_tense = gpt_model_corruptor.get_chatgpt_response(prompt_template_tense)
    print('ANSWER TENSE', answer_tense)
    if ',' in answer_tense:
        answer_all += answer_tense.split(',')
    
    prompt_template_negation = f'Dialogue: {dlg}\nFacts: {atomic_facts}\nSummary: {summary}\n{instruction}\nAnswer:'
    answer_negation = gpt_model_corruptor.get_chatgpt_response(prompt_template_negation)
    print('ANSWER NEGATION', answer_negation)
    if ',' in answer_tense:
        answer_all += answer_negation.split(',')
        
    answer_all = ','.join(answer_all)
    return answer_all

def get_generic_answer(gpt_model_corruptor, dlg, atomic_facts, summary, answer):
    instruction = f'What alternative words or phrases to "{answer}" can be used to fill in the <BLANK> of the summary that would make the summary inconsistent or misleading with respect to the dialogue or facts.'
    prompt_template = f'Dialogue: {dlg}\nFacts: {atomic_facts}\nSummary: {summary}\n{instruction}\nAnswer:'
    answer = gpt_model_corruptor.get_chatgpt_response(prompt_template)
    return answer
    
def check_sentence_validity(gpt_model_sentence_checker, text):
    instruction = "Correct the sentence for any grammatical errors without replacing any existing words"
    prompt_template = f'{instruction}\nSummary: {text}\nCorrected Summary:'
    answer = gpt_model_sentence_checker.get_chatgpt_response(prompt_template)
    return answer

def check_entailment(gpt_model_nli, dlg, facts, text):
    instruction = "Does the dialogue or facts entail the sentence?"
    prompt_template = f'{instruction}\nDialogue: {dlg}\nFacts: {atomic_facts}\nSentence: {text}\nAnswer(yes or no):'
    answer = gpt_model_nli.get_chatgpt_response(prompt_template)
    return answer

In [6]:
# def 

In [7]:
# mask_all_subjobj(
gpt_model_corruptor = GPTInference()
gpt_model_sentence_checker = GPTInference()
gpt_model_atomic = GPTInference()
gpt_model_nli = GPTInference()
text = df['Summary'].values[10]
dlg = df['Dialogue'].values[10]
# subjobj_blanks = mask_all_subjobj(text)

In [69]:
# instruction_intrinsic = "Generate a summary that misinterprets the conversation snippet"
# # instruction_intrinsic = "Generate a summary of the conversation snippet below and take any creative liberties"
# # instruction = "Formulate a summary and use creative liberties to make up extra information that is not found in the dialogue snippet."
# # instruction = "An inconsistent summary contains information not supported by the dialogue snippet. Generate an inconsistent summary of the dialogue snippet."
# prompt = f'{instruction}\n{dlg}'
# print(prompt)
# GPTInference().get_chatgpt_response(prompt)
# ### Check if inconsistent summaries just negates 

In [12]:
import re, string
texts = get_atomic_facts_gpt(gpt_model_atomic, 'gpt-4-32k-0613', dlg, 'dialogue')
texts = texts.split('\n')
texts = [re.sub('[1-9]', '', each) for each in texts]
texts = [each.strip(string.punctuation).strip() for each in texts]


Segment the following dialogue into important facts
Dialogue: Greg: Hi, honey. I need to stay after hours :-(
Betsy: Again?
Greg: I'm sorry!
Betsy: What about Johnny?
Greg: Well, could you pick him up? 
Betsy: What if I can't?
Greg: Betsy?
Betsy: What if I can't?
Greg: Can't you, really?
Betsy: I can't. Today I need to work long hours as well. Tuesdays are your days in the kindergarten.
Greg: Talk to you later. I'll see what I can do.
Betsy: You'd better think of something.
Greg: Oh. Just stop it now.


In [14]:
texts

['Greg has to stay at work after hours',
 'Greg asks Betsy if she can pick up Johnny',
 'Betsy initially questions her ability to pick up Johnny',
 'It is revealed that Betsy also has to work long hours',
 "Tuesdays are typically Greg's days to look after Johnny at kindergarten",
 'Greg promises Betsy that he will try to figure out a solution',
 'Betsy insists that Greg come up with a plan for Johnny']

In [15]:
model = "gpt-3.5-turbo-16k-0613"
texts = get_atomic_facts_gpt(gpt_model_atomic, model, dlg, 'dialogue')
texts = texts.split('\n')
texts = [re.sub('[1-9]', '', each) for each in texts]
texts = [each.strip(string.punctuation).strip() for each in texts]
texts

Segment the following dialogue into important facts
Dialogue: Greg: Hi, honey. I need to stay after hours :-(
Betsy: Again?
Greg: I'm sorry!
Betsy: What about Johnny?
Greg: Well, could you pick him up? 
Betsy: What if I can't?
Greg: Betsy?
Betsy: What if I can't?
Greg: Can't you, really?
Betsy: I can't. Today I need to work long hours as well. Tuesdays are your days in the kindergarten.
Greg: Talk to you later. I'll see what I can do.
Betsy: You'd better think of something.
Greg: Oh. Just stop it now.


['Greg needs to stay after hours',
 'Betsy is annoyed that Greg needs to stay after hours again',
 'Greg apologizes',
 'Greg asks Betsy to pick up Johnny',
 'Betsy questions if she can pick up Johnny',
 "Betsy says she can't pick up Johnny because she also needs to work long hours",
 'Betsy reminds Greg that Tuesdays are his days in the kindergarten',
 'Greg says he will see what he can do',
 'Betsy tells Greg to think of something',
 '0. Greg tells Betsy to stop']

In [18]:
model = "gpt-4-32k-0314"
texts = get_atomic_facts_gpt(gpt_model_atomic, model, dlg, 'dialogue')
texts = texts.split('\n')
texts = [re.sub('[1-9]', '', each) for each in texts]
texts = [each.strip(string.punctuation).strip() for each in texts]
texts

Segment the following dialogue into important facts
Dialogue: Greg: Hi, honey. I need to stay after hours :-(
Betsy: Again?
Greg: I'm sorry!
Betsy: What about Johnny?
Greg: Well, could you pick him up? 
Betsy: What if I can't?
Greg: Betsy?
Betsy: What if I can't?
Greg: Can't you, really?
Betsy: I can't. Today I need to work long hours as well. Tuesdays are your days in the kindergarten.
Greg: Talk to you later. I'll see what I can do.
Betsy: You'd better think of something.
Greg: Oh. Just stop it now.


['Greg needs to stay after hours at work',
 'Betsy is unable to pick up Johnny due to her own long work hours',
 "Tuesdays are usually Greg's days to handle kindergarten responsibilities",
 'Greg plans to find a solution for picking up Johnny']

In [73]:
masked_summaries = mask_all_errtypes(texts[0])

[(0, 'Greg', 'subjobj'), (1, 'needs', 'predicate'), (3, 'stay', 'predicate'), (4, 'after', 'circumstance'), (5, 'hours', 'subjobj'), (6, 'at', 'circumstance'), (7, 'work', 'subjobj')]


In [88]:
masked_summaries

[('<BLANK> needs to stay after hours at work', 'Greg', 'subjobj'),
 ('Greg needs to stay after <BLANK> at work', 'hours', 'subjobj'),
 ('Greg needs to stay after hours at <BLANK>', 'work', 'subjobj'),
 ('Greg <BLANK> to stay after hours at work', 'needs', 'predicate'),
 ('Greg needs to <BLANK> after hours at work', 'stay', 'predicate'),
 ('Greg needs to stay <BLANK> hours at work', 'after', 'circumstance'),
 ('Greg needs to stay after hours <BLANK> work', 'at', 'circumstance')]

In [124]:
text = masked_summaries[5][0]
answer = masked_summaries[5][1]
instruction = f'What alternative words or phrases to "{answer}" can be used to fill in the <BLANK> of the summary that would make the summary inconsistent or misleading with respect to the dialogue or facts.'
source = '\n'.join(texts)
prompt_template = f'Dialogue: {dlg}\nFacts: {source}\nSummary: {text}\n{instruction}\nAnswer:'
print(prompt_template)
answer = gpt_model_corruptor.get_chatgpt_response(prompt_template)
answer

Dialogue: Greg: Hi, honey. I need to stay after hours :-(
Betsy: Again?
Greg: I'm sorry!
Betsy: What about Johnny?
Greg: Well, could you pick him up? 
Betsy: What if I can't?
Greg: Betsy?
Betsy: What if I can't?
Greg: Can't you, really?
Betsy: I can't. Today I need to work long hours as well. Tuesdays are your days in the kindergarten.
Greg: Talk to you later. I'll see what I can do.
Betsy: You'd better think of something.
Greg: Oh. Just stop it now.
Facts: Greg needs to stay after hours at work
Betsy is also working long hours
It's Greg's turn to pick up Johnny from kindergarten on Tuesdays
Both Greg and Betsy are unable to pick up Johnny
Greg will try to find a solution
Summary: Greg needs to stay <BLANK> hours at work
What alternative words or phrases to "after" can be used to fill in the <BLANK> of the summary that would make the summary inconsistent or misleading with respect to the dialogue or facts.
Answer:


'before, regular, fewer, short'

In [64]:
doc = nlp('Lucas is waiting at the train station. He will be there by himself')
for idx, token in enumerate(doc):
        # append = False
        print(token.pos_)

PROPN
AUX
VERB
ADP
DET
NOUN
NOUN
PUNCT
PRON
AUX
VERB
ADV
ADP
PRON


In [63]:
# import nltk
# doc = nlp("And now for something completely different after dinner she said she can't")
# for idx, token in enumerate(doc):
#         # append = False
#         print(token.ent_)

AttributeError: 'spacy.tokens.token.Token' object has no attribute 'ent_'

In [60]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Lucas is waiting at the train station")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Lucas 0 5 GPE


In [55]:
subjobj_blanks = mask_all_errtypes(texts[1])

[(0, 'Betsy', 'subjobj'), (1, 'also', 'circumstance'), (2, 'needs', 'predicate'), (4, 'work', 'predicate'), (6, 'hours', 'subjobj'), (7, 'on', 'circumstance'), (10, 'day', 'subjobj')]


In [56]:
subjobj_blanks

[('<BLANK> also needs to work long hours on the same day', 'Betsy', 'subjobj'),
 ('Betsy also needs to work long <BLANK> on the same day', 'hours', 'subjobj'),
 ('Betsy also needs to work long hours on the same <BLANK>', 'day', 'subjobj'),
 ('Betsy <BLANK> needs to work long hours on the same day',
  'also',
  'circumstance'),
 ('Betsy also needs to work long hours <BLANK> the same day',
  'on',
  'circumstance'),
 ('Betsy also <BLANK> to work long hours on the same day',
  'needs',
  'predicate'),
 ('Betsy also needs to <BLANK> long hours on the same day',
  'work',
  'predicate')]

In [57]:
import string 
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = stopwords.words('english')


def make_corrupted_summary(text_blanks, facts):
    corrupted_err_type = {}
    for blanked_text, answer, err_type in text_blanks:
        if err_type == 'subjobj':
            corrupted_answer = get_generic_answer(gpt_model_corruptor, dlg, facts, blanked_text, answer)
            # print(blanked_text, err_type)
            # print(answer)
            # print(corrupted_answer)
            # print('====')
        if err_type == 'predicate':
            corrupted_answer = get_verb_answer(gpt_model_corruptor, dlg, facts, blanked_text, answer)
            print(blanked_text, err_type)
            print(answer)
            print(corrupted_answer)
            print('====')
        else:
            
    
        
        options = corrupted_answer.split(',')
        # print(options)
        corrupted_facts = []
        for option in options:
            corrupted_summary = blanked_text.replace('<BLANK>', option.strip())
            corrected_summary = check_sentence_validity(gpt_model_sentence_checker, corrupted_summary)
            
            corrected_summary = [each.strip(string.punctuation) for each in corrected_summary.split(' ')]
            corrupted_summary = [each.strip(string.punctuation) for each in corrupted_summary.split(' ')]
            
            word_constraints = [each for each in blanked_text.split(' ') if each != '<BLANK>' and each not in stop_words]
            constrain_overlap = set(word_constraints) - set(corrected_summary)
            replaced_word = ' '.join([each for each in corrected_summary if each not in blanked_text.split(' ')])
    
            corrected_summary = ' '.join(corrected_summary)
            corrupted_summary = ' '.join(corrupted_summary)
        
            if not constrain_overlap and replaced_word: 
                entailment = check_entailment(gpt_model_nli, dlg, facts, corrected_summary)
                # print(blanked_text)
                # print(corrected_summary, entailment)
                # print(replaced_word)
                # print('----')
                if entailment == 'no':
                    if err_type not in corrupted_err_type:
                        corrupted_err_type[err_type] = []
                    corrupted_err_type[err_type] += [(corrected_summary, replaced_word)]
    return corrupted_err_type

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sanjana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
atomic_fact_map = {}
facts = '\n'.join(texts)
for atomic_fact in texts:
    atomic_fact_blanks = mask_all_errtypes(atomic_fact)
    atomic_fact_corrupted = make_corrupted_summary(atomic_fact_blanks, facts)
    atomic_fact_map[atomic_fact] = atomic_fact_corrupted

[(0, 'Greg', 'subjobj'), (1, 'needs', 'predicate'), (3, 'stay', 'predicate'), (4, 'after', 'circumstance'), (5, 'hours', 'subjobj'), (6, 'at', 'circumstance'), (7, 'work', 'subjobj')]
ANSWER VERB has, wants, is required, is forced
ANSWER TENSE needed, will need, might need, has needed, requiring
ANSWER NEGATION doesn't need, refuses, declines, avoids, has no need, is not required, denies needing, is unwilling
Greg <BLANK> to stay after hours at work predicate
needs
has, wants, is required, is forced,needed, will need, might need, has needed, requiring,doesn't need, refuses, declines, avoids, has no need, is not required, denies needing, is unwilling
====
ANSWER VERB work, remain, stick around, linger, spend time
ANSWER TENSE stayed, staying, will stay, was staying, has stayed, had stayed, will have stayed, would stay, would have stayed, is staying, might stay, may stay
ANSWER NEGATION leave, depart, go away, avoid staying, exit, vacate, delay
Greg needs to <BLANK> after hours at work p


KeyboardInterrupt



In [815]:
atomic_fact_map

{'Greg needs to stay after hours at work': {'subjobj': [('Johnny needs to stay after hours at work',
    'Johnny'),
   ('Greg needs to stay after Johnny at work', 'Johnny'),
   ('Greg needs to stay after Betsy at work', 'Betsy')],
  'verb': [('Greg prefers to stay after hours at work', 'prefers'),
   ('Greg volunteers to stay after hours at work', 'volunteers'),
   ("Greg doesn't need to stay after hours at work", "doesn't need"),
   ('Greg has no need to stay after hours at work', 'has no need'),
   ("Greg isn't required to stay after hours at work", "isn't required"),
   ('Greg refuses to stay after hours at work', 'refuses'),
   ("Greg can't stay after hours at work", "can't"),
   ("Greg won't stay after hours at work", "won't"),
   ("Greg shouldn't stay after hours at work", "shouldn't"),
   ('Greg declines to stay after hours at work', 'declines'),
   ('Greg escapes to stay after hours at work', 'escapes'),
   ("Greg won't have to stay after hours at work", "won't have"),
   ('Gre

In [808]:
from random import sample

def get_single_err_samples(num_single_err_samples):
# num_single_err_samples = 4
    all_summary_sents = list(atomic_fact_map.keys())
    
    picked_sent_idx = sample([idx for idx in range(0, len(all_summary_sents))], num_single_err_samples)
    
    # corrupted_pick_sent_span = []
    corrupted_summ_sent_spans = []
    for pidx in picked_sent_idx:
        picked_sent = all_summary_sents[pidx]
        # summary = all_summary_sents[:pidx] + []
        for k , v in atomic_fact_map[picked_sent].items():
            # print(atomic_fact_map[picked_sent])
            corrupted_pick_sent_span = sample(v, 1)[0]
            print(pidx, 
            corrupted_summary = all_summary_sents[:pidx] + [corrupted_pick_sent_span[0]] + all_summary_sents[pidx:]
            corrupted_summ_sent_spans.append(('. '.join(corrupted_summary), 
                                              [(corrupted_pick_sent_span[0], 
                                              corrupted_pick_sent_span[1])]))
    return corrupted_summ_sent_spans



In [809]:
corrupted_summ_sent_spans = get_single_err_samples(4)

In [810]:
corrupted_summ_sent_spans[0][1]

[('Both Greg and Betsy are unable to work long hours with Johnny that day',
  'work long hours with')]

In [811]:
ic_examples = []
for summ_spans in corrupted_summ_sent_spans:
    list_errors = '\n\n'.join([f'({each[0]} , {each[1]})' for each in summ_spans[1]])
    ic_ex = f'Summary: {summ_spans[0]}\nInconsistent sentence/spans:{list_errors}'
    # print(ic_ex)
    ic_examples += [ic_ex]

In [812]:
inconsistent_instruction = 'Decide whether the summary is consistent with the dialogue. Note consistency means all the information in the summary can be found or inferred from the dialogue. A consistent summary does not contain contradictory or new information. List the inconsistent sentence and spans'
ic_examples = '\n\n'.join(ic_examples)

prompt_template = f'{inconsistent_instruction}\nDialogue: {dlg}\n\n{ic_examples}\n\nSummary:{text}\nInconsistent sentence/spans:'



In [813]:
print(prompt_template)

Decide whether the summary is consistent with the dialogue. Note consistency means all the information in the summary can be found or inferred from the dialogue. A consistent summary does not contain contradictory or new information. List the inconsistent sentence and spans
Dialogue: Greg: Hi, honey. I need to stay after hours :-(
Betsy: Again?
Greg: I'm sorry!
Betsy: What about Johnny?
Greg: Well, could you pick him up? 
Betsy: What if I can't?
Greg: Betsy?
Betsy: What if I can't?
Greg: Can't you, really?
Betsy: I can't. Today I need to work long hours as well. Tuesdays are your days in the kindergarten.
Greg: Talk to you later. I'll see what I can do.
Betsy: You'd better think of something.
Greg: Oh. Just stop it now.

Summary: Greg needs to stay after hours at work. Betsy is also working long hours that day. It is Greg's responsibility to pick up Johnny from kindergarten on Tuesdays. Both Greg and Betsy are unable to work long hours with Johnny that day. Both Greg and Betsy are unab

In [793]:
GPTInference().get_chatgpt_response(prompt_template)

'(None)'